In [31]:
import pandas as pd
import numpy as np

In [32]:
books = pd.read_csv("https://github.com/gdv/foundationsCS/raw/master/progetti/2021/Books.csv.gz", sep = ';', encoding = 'latin-1', low_memory = False)
books_rating = pd.read_csv("https://github.com/gdv/foundationsCS/raw/master/progetti/2021/Book-Ratings.csv.gz", sep = ';', encoding = 'latin-1', low_memory = False)
users = pd.read_csv("https://github.com/gdv/foundationsCS/raw/master/progetti/2021/Users.csv.gz",  sep = ';', encoding = 'latin-1', low_memory = False)
goodbooks = pd.read_csv("https://github.com/gdv/foundationsCS/raw/master/progetti/2021/goodbooks.csv.gz", encoding = 'latin-1', low_memory = False)
goodbooks_ratings = pd.read_csv("https://github.com/gdv/foundationsCS/raw/master/progetti/2021/goodbooks-ratings.csv.gz", encoding = 'latin-1', low_memory = False)

# 1 - Normalize the location field of Users dataset, splitting into city, region, country.


In [33]:
users[['City', 'Region', 'Country' ]]= users['Location'].str.split(',', n = 2,expand=True )
#splitto la colonna location tramite .str.split e dividendola per la virgola 
#il 2 sta a significare: Limitare il numero di suddivisioni nell'output. 
#Se non viene fornito, il valore predefinito è -1, il che significa che non c'è limite.

users.head()

,User-ID,Location,Age,City,Region,Country
0,1,"nyc, new york, usa",NaN,nyc,new york,usa
1,2,"stockton, california, usa",18.0,stockton,california,usa
2,3,"moscow, yukon territory, russia",NaN,moscow,yukon territory,russia
3,4,"porto, v.n.gaia, portugal",17.0,porto,v.n.gaia,portugal
4,5,"farnborough, hants, united kingdom",NaN,farnborough,hants,united kingdom


# 2. For each book in the Books dataset, compute its average rating.

In [61]:
merged_books = pd.merge(books, books_rating, on = 'ISBN', how = 'left').groupby(['Book-Title','ISBN'], as_index = False).agg(Average_rating=('Book-Rating','mean'))
#Sono stati uniti i dataset usando come vincolo d'integrità referenziale 'ISBN', dopo aver raggruppato per 'ISBN' e 'Book-Title' è stata calcolata la media del rating per ogni libro.
merged_books.head()

,Book-Title,ISBN,Average_rating
0,A Light in the Storm: The Civil War Diary of ...,0590567330,2.25
1,Always Have Popsicles,0964147726,0.00
2,Apple Magic (The Collector's series),0942320093,0.00
3,"Ask Lily (Young Women of Faith: Lily Series, ...",0310232546,8.00
4,Beyond IBM: Leadership Marketing and Finance ...,0962295701,0.00


# 3-For each book in the GoodBooks dataset, compute its average rating

In [35]:
pd.set_option('display.max_columns', None)
#non tutte le colonne vengono mostrate essendo tante, in questo modo si può avere un quadro generale 
goodbooks.head(3)

,book_id,goodreads_book_id,best_book_id,work_id,books_count,isbn,isbn13,authors,original_publication_year,original_title,title,language_code,average_rating,ratings_count,work_ratings_count,work_text_reviews_count,ratings_1,ratings_2,ratings_3,ratings_4,ratings_5,image_url,small_image_url
0,1,2767052,2767052,2792775,272,439023483,9.780439e+12,Suzanne Collins,2008.0,The Hunger Games,"The Hunger Games (The Hunger Games, #1)",eng,4.34,4780653,4942365,155254,66715,127936,560092,1481305,2706317,https://images.gr-assets.com/books/1447303603m...,https://images.gr-assets.com/books/1447303603s...
1,2,3,3,4640799,491,439554934,9.780440e+12,"J.K. Rowling, Mary GrandPrÃ©",1997.0,Harry Potter and the Philosopher's Stone,Harry Potter and the Sorcerer's Stone (Harry P...,eng,4.44,4602479,4800065,75867,75504,101676,455024,1156318,3011543,https://images.gr-assets.com/books/1474154022m...,https://images.gr-assets.com/books/1474154022s...
2,3,41865,41865,3212258,226,316015849,9.780316e+12,Stephenie Meyer,2005.0,Twilight,"Twilight (Twilight, #1)",en-US,3.57,3866839,3916824,95009,456191,436802,793319,875073,1355439,https://images.gr-assets.com/books/1361039443m...,https://images.gr-assets.com/books/1361039443s...


In [36]:
goodbooks[['isbn','original_title','average_rating']]
#stampiamo le colonne che ci interessano
#non c'è bisogno di eseguire nessun calcolo o merge visto che la colonna di interesse è già presente nel dataset

,isbn,original_title,average_rating
0,439023483,The Hunger Games,4.34
1,439554934,Harry Potter and the Philosopher's Stone,4.44
2,316015849,Twilight,3.57
3,61120081,To Kill a Mockingbird,4.25
4,743273567,The Great Gatsby,3.89
...,...,...,...
94,375751513,The Picture of Dorian Gray,4.06
95,345803507,Fifty Shades Freed,3.88
96,393970124,Dracula,3.98
97,307269981,Flickan som lekte med elden,4.22


In [37]:
#Seconda soluzione
goodbooks.groupby(['book_id'], as_index = False)[['average_rating']].mean()

,book_id,average_rating
0,1,4.34
1,2,4.44
2,3,3.57
3,4,4.25
4,5,3.89
...,...,...
94,95,4.06
95,96,3.88
96,97,3.98
97,98,4.22


# 4-Merge together all rows sharing the same book title, author and publisher. We will call the resulting dataset "merged books". The books that have not been merged together will not appear in merged books.


In [38]:
books2 = books.groupby(['Book-Title','Book-Author','Publisher'], as_index = False).count()
#raggruppo i libri che hanno lo stesso titolo, autorie e casa editrice contando
#anche quante volte si vi sono nel db 
books2.head(2)

,Book-Title,Book-Author,Publisher,ISBN,Year-Of-Publication,Image-URL-S,Image-URL-M,Image-URL-L
0,A Light in the Storm: The Civil War Diary of ...,Karen Hesse,Hyperion Books for Children,1,1,1,1,1
1,Always Have Popsicles,Rebecca Harvin,Rebecca L. Harvin,1,1,1,1,1


In [39]:
merged_books = pd.DataFrame(books2[books2['ISBN']>=2])[['Book-Title', 'Book-Author','Publisher', 'ISBN']]
#da books2 prendo in considerazione chi ha avuto almeno 2 edizioni mostrando poi 
#solo le colonne di interesse 

merged_books.head()

,Book-Title,Book-Author,Publisher,ISBN
41,!%@ (A Nutshell handbook),Donnalyn Frey,O'Reilly,2
59,'A Hell of a Place to Lose a Cow': An American...,Tim Brookes,National Geographic,2
255,"10,000 dreams interpreted: A dictionary of dreams",Gustavus Hindman Miller,Barnes &amp; Nobles Books,2
465,101 Famous Poems,Roy J. Cook,McGraw-Hill/Contemporary Books,3
713,15 Houseplants Even You Can't Kill,Joe Elder,Berkley Pub Group,2


# 5. For each book in merged books compute its average rating.


In [40]:
books_rating_avg = books_rating.groupby('ISBN', as_index = True).agg(avg_rating=('Book-Rating','mean'))
#raggruppo per ISBN e ne calcolo la media con la funzione di aggregazione 

books_rating_avg.head(2)

,avg_rating
ISBN,
0330299891,3.0
0375404120,1.5


In [63]:
books_dubplicati = books[books.duplicated(subset =['Book-Title', 'Book-Author', 'Publisher'], keep = False)].sort_values(['Book-Title', 'Book-Author', 'Publisher'])
#uso la funzione duplicati per ottenere un risultato booleano e tenendo keep = false cosi facendo non eliminiamo i duplicati 
#vado poi a ordinare le colonne 
books_dubplicati[['ISBN','Book-Title']].head(6)


,ISBN,Book-Title
75637,1565920465,!%@ (A Nutshell handbook)
156341,1565920317,!%@ (A Nutshell handbook)
140618,0792276833,'A Hell of a Place to Lose a Cow': An American...
158204,0792277295,'A Hell of a Place to Lose a Cow': An American...
99327,1566196256,"10,000 dreams interpreted: A dictionary of dreams"
179167,1566196264,"10,000 dreams interpreted: A dictionary of dreams"


In [42]:
merged_books_rating = pd.merge(books_dubplicati,books_rating_avg, on =  'ISBN', how= 'left')
# mergio con una left join i risultati trovati sopra con la chiave ISBN


merged_books_rating[['ISBN','Book-Title','Book-Author','Publisher','avg_rating']]
#estraggo solo le colonne che mi interessa vedere 

,ISBN,Book-Title,Book-Author,Publisher,avg_rating
0,1565920465,!%@ (A Nutshell handbook),Donnalyn Frey,O'Reilly,0.000000
1,1565920317,!%@ (A Nutshell handbook),Donnalyn Frey,O'Reilly,6.000000
2,0792276833,'A Hell of a Place to Lose a Cow': An American...,Tim Brookes,National Geographic,0.000000
3,0792277295,'A Hell of a Place to Lose a Cow': An American...,Tim Brookes,National Geographic,3.400000
4,1566196256,"10,000 dreams interpreted: A dictionary of dreams",Gustavus Hindman Miller,Barnes &amp; Nobles Books,7.250000
...,...,...,...,...,...
9949,1569317674,Zoids Chaotic Century (Zoids: Chaotic Century ...,Michiro Ueyama,Viz Comics,10.000000
9950,1569317666,Zoids Chaotic Century (Zoids: Chaotic Century ...,Michiro Ueyama,Viz Comics,10.000000
9951,1569317658,Zoids Chaotic Century (Zoids: Chaotic Century ...,Michiro Ueyama,Viz Comics,10.000000
9952,3492218091,Ã?Ã?berleben in der WÃ?ÃÂ¼ste Danakil.,RÃ?ÃÂ¼diger Nehberg,Piper,5.000000


# 6. For each book in merged books compute the minimum and maximum of the average ratings over all corresponding books in the books dataset.

In [66]:
merged_books_rating_final_max = merged_books_rating.groupby(['Book-Title', 'Book-Author', 'Publisher'], as_index = False).agg({'avg_rating': ['min','max']}).round(2)
#raggruppo per titolo e aggrego ma deia raiting per valore massimo e minimo 

merged_books_rating_final_max.head(6)

Book-Title              Book-Author  \
                                                                                
0                          !%@ (A Nutshell handbook)            Donnalyn Frey   
1  'A Hell of a Place to Lose a Cow': An American...              Tim Brookes   
2  10,000 dreams interpreted: A dictionary of dreams  Gustavus Hindman Miller   
3                                   101 Famous Poems              Roy J. Cook   
4                 15 Houseplants Even You Can't Kill                Joe Elder   
5                                 158 POUND MARRIAGE              John Irving   

                        Publisher avg_rating        
                                         min   max  
0                        O'Reilly       0.00  6.00  
1             National Geographic       0.00  3.40  
2       Barnes &amp; Nobles Books       6.67  7.25  
3  McGraw-Hill/Contemporary Books       0.00  5.00  
4               Berkley Pub Group       0.00  0.00  
5                          Pocket       1.33  3.50

# 7 - For each book in goodbooks, compute the list of its authors. Assuming that the number of reviews with a text (column work_text_reviews_count)is split equally among all authors, find for each authors the total number of reviews with a text. We will call this quantity the shared number of reviews with a text

In [44]:
goodbooks['autori_splitted'] = goodbooks['authors'].str.split(',')
#creo una nuova colonna splittando gli autori con la ',' come divisore creando una lista di autori per ogni riga
goodbooks[['authors','autori_splitted']].head(4)

,authors,autori_splitted
0,Suzanne Collins,[Suzanne Collins]
1,"J.K. Rowling, Mary GrandPrÃ©","[J.K. Rowling, Mary GrandPrÃ©]"
2,Stephenie Meyer,[Stephenie Meyer]
3,Harper Lee,[Harper Lee]


In [67]:
goodbooks['shared_number_of_reviews_with_a_text'] = goodbooks['work_text_reviews_count'] / goodbooks['autori_splitted'].apply(len)
#E' stato calcolato il rapporto tra il numero di reviews e la lunghezza di ogni lista che rappresenta il numero di autori per ogni riga.
goodbooks[['autori_splitted','work_text_reviews_count', 'shared_number_of_reviews_with_a_text']].head(2)

,autori_splitted,work_text_reviews_count,shared_number_of_reviews_with_a_text
0,[Suzanne Collins],155254,155254.0
1,"[J.K. Rowling, Mary GrandPrÃ©]",75867,37933.5


In [46]:
goodbooks_exploded = goodbooks.explode('autori_splitted')
#Con il comando .explode abbiamo trasformato ogni elemento di un elenco in una riga, in modo che per ogni riga ci sia solo un autore.
goodbooks_exploded[['autori_splitted']].head(3)

,autori_splitted
0,Suzanne Collins
1,J.K. Rowling
1,Mary GrandPrÃ©


In [70]:
goodbooks_exploded.groupby(['autori_splitted'], as_index=False).sum('shared_number_of_reviews_with_a_text')[['autori_splitted', 'shared_number_of_reviews_with_a_text']].round(2).head(3)
# raggruppo per autori e ne sommo le reviews


,autori_splitted,shared_number_of_reviews_with_a_text
0,Alan R. Clarke,27890.50
1,B.M. Mooyaart-Doubleday,6941.67
2,Bernard Knox,1620.20


# 8 - For each year of publication, determine the author that has the largest value of the shared number of reviews with a text.

In [48]:
ex_8 = goodbooks_exploded.groupby(['original_publication_year', 'autori_splitted'], as_index=False)[['shared_number_of_reviews_with_a_text']].sum()
#Raggruppo per anni e autori e trovo la somma dei risultati ottenuti al punto precendente 
ex_8.head(4)

,original_publication_year,autori_splitted,shared_number_of_reviews_with_a_text
0,-720.0,Bernard Knox,1620.2
1,-720.0,E.V. Rieu,1620.2
2,-720.0,FrÃ©dÃ©ric Mugler,1620.2
3,-720.0,Robert Fagles,1620.2


In [49]:
ex_8.rename(columns= {'shared_number_of_reviews_with_a_text':'somma_shared'}, inplace=True)
#cambio il nome alla colonna 

ex_8.groupby('original_publication_year').max()
#per ogni anno trovo il valore massimo 

,autori_splitted,somma_shared
original_publication_year,,
-720.0,Homer,1620.200000
1595.0,William Shakespeare,7389.000000
1811.0,Jane Austen,3842.000000
1813.0,Jane Austen,49152.000000
1818.0,Mary Wollstonecraft Shelley,6664.333333
1847.0,Emily BrontÃ«,15606.000000
1859.0,Charles Dickens,4364.333333
1868.0,Louisa May Alcott,17090.000000
1884.0,Mark Twain,4149.333333


# 9 - Assuming that there are no errors in the ISBN fields,find the books in both datasets,and compute the difference of average rating according to the ratings and the goodratings datasets

In [50]:
avg_books = pd.merge(books, books_rating, on = 'ISBN', how = 'left').groupby(['Book-Title','ISBN'], as_index = False).agg(Average_rating=('Book-Rating','mean'))
#Sono stati uniti i dataset usando come vincolo d'integrità referenziale 'ISBN', dopo aver raggruppato per 'ISBN' e 'Book-Title' è stata calcolata la media del rating per ogni libro.
avg_books.head(3)

,Book-Title,ISBN,Average_rating
0,A Light in the Storm: The Civil War Diary of ...,0590567330,2.25
1,Always Have Popsicles,0964147726,0.00
2,Apple Magic (The Collector's series),0942320093,0.00


In [51]:
books_both_datasets = pd.merge(avg_books, goodbooks, left_on='ISBN', right_on='isbn')
#faccio un merge tra 'books' e 'good_books' per ottenere tutti i libri che compaiono in entrambi i dataset

del books_both_datasets['isbn']
books_both_datasets.rename(columns={'average_rating': 'Goodbook_Average_Rating'}, inplace=True)
# elimino una delle due colonne superflua e rinomino average_rating in Goodbook_Average_Rating

books_both_datasets['Differenza'] = books_both_datasets['Goodbook_Average_Rating'] - books_both_datasets['Average_rating']
#Calcolo la differenza fra i due punteggi.
books_both_datasets[['ISBN', 'Book-Title','Goodbook_Average_Rating','Average_rating', 'Differenza']].head(3)

,ISBN,Book-Title,Goodbook_Average_Rating,Average_rating,Differenza
0,014028009X,Bridget Jones's Diary,3.75,3.751852,-0.001852
1,043965548X,Harry Potter and the Prisoner of Azkaban (Harr...,4.53,3.533333,0.996667
2,1400032717,The Curious Incident of the Dog in the Night-T...,3.85,4.813187,-0.963187


# 10 - Split the users dataset according to the age. One dataset contains the users with unknown age, one with age 0-14, one with age 15-24, one with age 25-34, and so on.

In [71]:
users['age_group'] = pd.cut(x = users['Age'], bins = [0, 14, 24, 34, 44, 54, 64, 74, 84,94,100,1000], labels=['0-14','15-24','25-34','35-44','45-54','55-64','65-74','75-84','85-94','95-100','over']) 
# usando la funzine pd.cut e' possibile convertire l'eta in gruppi di eta' 
# per bins si intendono i bordi che vogliamo porre 
# per LABELS Specifica le etichette per i bin restituiti
users['age_group']

0           NaN
1         15-24
2           NaN
3         15-24
4           NaN
          ...  
278853      NaN
278854    45-54
278855      NaN
278856      NaN
278857      NaN
Name: age_group, Length: 278858, dtype: category
Categories (11, object): ['0-14' < '15-24' < '25-34' < '35-44' ... '75-84' < '85-94' < '95-100' < 'over']

In [73]:
is_null = users[users['age_group'].isnull()]
#creo un dataset con età mancante 
is_null[['age_group']].head(5)

,age_group
0,NaN
2,NaN
4,NaN
6,NaN
7,NaN


In [54]:
u_0_14 = users[users['age_group']== '0-14'] 
u_14_24 = users[users['age_group']== '14-24'] 
u_25_34 = users[users['age_group']== '25-34'] 
u_35_44 = users[users['age_group']== '35-44'] 
u_45_54 = users[users['age_group']== '45-54'] 
u_55_64 = users[users['age_group']== '55-64'] 
u_65_74 = users[users['age_group']== '65-74'] 
u_75_84 = users[users['age_group']== '75-84'] 
u_85_94 = users[users['age_group']== '85-94'] 
u_95_100 = users[users['age_group']== '95-100'] 
over_users = users[users['age_group']=='over']
over_users
#i Vari dataset divisi per età
# la richiesta era di splittare il dataset nelle diverse classi di eta' 

,User-ID,Location,Age,City,Region,Country,age_group
1288,1289,"san jose, california, usa",103.0,san jose,california,usa,over
1322,1323,"milano, lombardia, italy",104.0,milano,lombardia,italy,over
1578,1579,"akure, ondo/nigeria, nigeria",231.0,akure,ondo/nigeria,nigeria,over
3084,3085,"zÃ¼rich, switzerland, switzerland",104.0,zÃ¼rich,switzerland,switzerland,over
3210,3211,"le mesnil saint denis, yvelines, france",119.0,le mesnil saint denis,yvelines,france,over
...,...,...,...,...,...,...,...
276047,276048,"wien, wien, austria",127.0,wien,wien,austria,over
276352,276353,"hillsdale, new york, usa",104.0,hillsdale,new york,usa,over
277107,277108,"quinto, ticino, switzerland",104.0,quinto,ticino,switzerland,over
277503,277504,"san diego, california, usa",103.0,san diego,california,usa,over


# 11 - Find the books that appear only in the goodbooks datasets.


In [55]:
ex_11 = goodbooks[goodbooks['isbn'].isin(books['ISBN'])==False]
#uso una condizione booleana e la do per falsa prendendo cosi tutti i libri che vi sono in goodbooks
# e scartando quelli che non trovo il books

ex_11[['isbn','original_title']].head(10)

,isbn,original_title
0,439023483,The Hunger Games
1,439554934,Harry Potter and the Philosopher's Stone
2,316015849,Twilight
3,61120081,To Kill a Mockingbird
4,743273567,The Great Gatsby
5,525478817,The Fault in Our Stars
6,618260307,The Hobbit or There and Back Again
7,316769177,The Catcher in the Rye
8,1416524797,Angels & Demons
9,679783261,Pride and Prejudice


# 12. Assuming that each pair (author, title) identifies a book, for each book find the number of times it appears in the books dataset. Which books appear the most times?

In [75]:
count = books.groupby(['Book-Author','Book-Title']).count().sort_values(by='ISBN',ascending=False)
#Ho raggruppato per prima l'autore e il titolo e poi usato la funzione count per contare le righe di ogni 'ISBN' ordinnando
#poi in modo decrescente 

count.head(1)
# per vedere quale era il libro più ridondande bastava stampare solo la prima riga visto che era state ordinate 

,,ISBN,Year-Of-Publication,Publisher,Image-URL-S,Image-URL-M,Image-URL-L
Book-Author,Book-Title,,,,,,
Louisa May Alcott,Little Women,21,21,21,21,21,21


# 13. Find the author with the highest average rating according to the goodbooks datasets.


In [57]:
goodbooks['average_rating'].max()
# trovo il valore massimo dell'average_rating

4.61

In [58]:
goodbooks[goodbooks['average_rating']==4.61][['authors','original_title','average_rating']]
#trovato il valore e raggruppato per autori trovo la riga corrispondente stampando autori,titolo e media

,authors,original_title,average_rating
24,"J.K. Rowling, Mary GrandPrÃ©",Harry Potter and the Deathly Hallows,4.61


In [59]:
#oppure 
goodbooks.iloc[goodbooks['average_rating'].idxmax()][['authors', 'average_rating']]

authors           J.K. Rowling, Mary GrandPrÃ©
average_rating                            4.61
Name: 24, dtype: object